In [1]:
!git clone https://github.com/google/flatbuffers.git
!cd flatbuffers && cmake -G "Unix Makefiles" -DCMAKE_BUILD_TYPE=Release && make -j

!cd flatbuffers/python && python3 setup.py install

Cloning into 'flatbuffers'...
remote: Enumerating objects: 4, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 18346 (delta 2), reused 2 (delta 2), pack-reused 18342
Receiving objects: 100% (18346/18346), 11.03 MiB | 7.84 MiB/s, done.
Resolving deltas: 100% (12745/12745), done.
-- The C compiler identification is GNU 7.5.0
-- The CXX compiler identification is GNU 7.5.0
-- Check for working C compiler: /usr/bin/cc
-- Check for working C compiler: /usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Check for working CXX compiler: /usr/bin/c++
-- Check for working CXX compiler: /usr/bin/c++ -- works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Looking for strtof_l
-- Looking for strtof_l - found
-- Lookin

In [2]:
!wget https://raw.githubusercontent.com/tensorflow/tensorflow/master/tensorflow/lite/schema/schema.fbs

--2020-09-15 14:22:38--  https://raw.githubusercontent.com/tensorflow/tensorflow/master/tensorflow/lite/schema/schema.fbs
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26964 (26K) [text/plain]
Saving to: ‘schema.fbs’

schema.fbs          100%[===================>]  26.33K  --.-KB/s    in 0.01s   

2020-09-15 14:22:39 (2.09 MB/s) - ‘schema.fbs’ saved [26964/26964]



In [3]:
!alias python=python3
!./flatbuffers/flatc --python ./schema.fbs

In [4]:
!wget https://raw.githubusercontent.com/google/mediapipe/master/mediapipe/models/palm_detection.tflite

--2020-09-15 14:22:39--  https://raw.githubusercontent.com/google/mediapipe/master/mediapipe/models/palm_detection.tflite
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7090152 (6.8M) [application/octet-stream]
Saving to: ‘palm_detection.tflite’

palm_detection.tfli 100%[===================>]   6.76M  21.4MB/s    in 0.3s    

2020-09-15 14:22:41 (21.4 MB/s) - ‘palm_detection.tflite’ saved [7090152/7090152]



In [1]:
import os
import numpy as np
from collections import OrderedDict
from tflite import Model

In [2]:
data = open("./palm_detection.tflite", "rb").read()
model = Model.Model.GetRootAsModel(data, 0)
subgraph = model.Subgraphs(0)
subgraph.Name()

b'handdetector_hourglass_short_2019_03_25_v0.tflite.no_meta'

In [3]:
def get_shape(tensor):
    return [tensor.Shape(i) for i in range(tensor.ShapeLength())]

for i in range(0, subgraph.TensorsLength()):
    tensor = subgraph.Tensors(i)
    print("%3d %30s %d %2d %s" % (i, tensor.Name(), tensor.Type(), tensor.Buffer(), 
                                  get_shape(tensor)))

  0                       b'input' 0  0 [1, 256, 256, 3]
  1               b'conv2d/Kernel' 0  1 [32, 3, 3, 3]
  2                 b'conv2d/Bias' 0  2 [32]
  3                      b'conv2d' 0  0 [1, 128, 128, 32]
  4                  b'activation' 0  0 [1, 128, 128, 32]
  5     b'depthwise_conv2d/Kernel' 0  3 [1, 3, 3, 32]
  6       b'depthwise_conv2d/Bias' 0  4 [32]
  7            b'depthwise_conv2d' 0  0 [1, 128, 128, 32]
  8             b'conv2d_1/Kernel' 0  5 [32, 1, 1, 32]
  9               b'conv2d_1/Bias' 0  6 [32]
 10                    b'conv2d_1' 0  0 [1, 128, 128, 32]
 11                         b'add' 0  0 [1, 128, 128, 32]
 12                b'activation_1' 0  0 [1, 128, 128, 32]
 13   b'depthwise_conv2d_1/Kernel' 0  7 [1, 3, 3, 32]
 14     b'depthwise_conv2d_1/Bias' 0  8 [32]
 15          b'depthwise_conv2d_1' 0  0 [1, 128, 128, 32]
 16             b'conv2d_2/Kernel' 0  9 [32, 1, 1, 32]
 17               b'conv2d_2/Bias' 0 10 [32]
 18                    b'conv2d_2' 0  0 

In [4]:
tensor_dict = {(subgraph.Tensors(i).Name().decode("utf8")): i 
               for i in range(subgraph.TensorsLength())}

In [5]:
parameters = {}
for i in range(subgraph.TensorsLength()):
    tensor = subgraph.Tensors(i)
    if tensor.Buffer() > 0:
        name = tensor.Name().decode("utf8")
        parameters[name] = tensor.Buffer()

len(parameters)

185

In [6]:
def get_weights(tensor_name):
    i = tensor_dict[tensor_name]
    tensor = subgraph.Tensors(i)
    buffer = tensor.Buffer()
    shape = get_shape(tensor)
    assert(tensor.Type() == 0)  # FLOAT32
    
    W = model.Buffers(buffer).DataAsNumpy()
    W = W.view(dtype=np.float32)
    W = W.reshape(shape)
    return W
W = get_weights("conv2d/Kernel")
b = get_weights("conv2d/Bias")
W.shape, b.shape

((32, 3, 3, 3), (32,))

In [7]:
import torch
from blazepalm import PalmDetector

In [8]:
net = PalmDetector()
net

PalmDetector(
  (backbone1): Sequential(
    (0): ConstantPad2d(padding=(0, 1, 0, 1), value=0.0)
    (1): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2))
    (2): ReLU(inplace=True)
    (3): ResBlock(
      (f): Sequential(
        (0): ResModule(
          (convs): Sequential(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32)
            (1): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1))
          )
          (act): ReLU(inplace=True)
        )
        (1): ResModule(
          (convs): Sequential(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32)
            (1): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1))
          )
          (act): ReLU(inplace=True)
        )
        (2): ResModule(
          (convs): Sequential(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32)
            (1): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1))
          )

In [9]:
probable_names = []
for i in range(0, subgraph.TensorsLength()):
    tensor = subgraph.Tensors(i)
    if tensor.Buffer() > 0 and tensor.Type() == 0:
        probable_names.append(tensor.Name().decode("utf-8"))
        
probable_names[:5]

['conv2d/Kernel',
 'conv2d/Bias',
 'depthwise_conv2d/Kernel',
 'depthwise_conv2d/Bias',
 'conv2d_1/Kernel']

In [10]:
convert = {}
i = 0
for name, params in net.state_dict().items():
    convert[name] = probable_names[i]
    i += 1

In [11]:
new_state_dict = OrderedDict()

for dst, src in convert.items():
    W = get_weights(src)
    print(dst, src, W.shape, net.state_dict()[dst].shape)

    if W.ndim == 4:
        if W.shape[0] == 1:
            W = W.transpose((3, 0, 1, 2))  # depthwise conv
        else:
          if "transpose" in src:
            W = W.transpose((3, 0, 1, 2)) # transpose
          else:
            W = W.transpose((0, 3, 1, 2))  # regular conv
    
    new_state_dict[dst] = torch.from_numpy(W)


backbone1.1.weight conv2d/Kernel (32, 3, 3, 3) torch.Size([32, 3, 3, 3])
backbone1.1.bias conv2d/Bias (32,) torch.Size([32])
backbone1.3.f.0.convs.0.weight depthwise_conv2d/Kernel (1, 3, 3, 32) torch.Size([32, 1, 3, 3])
backbone1.3.f.0.convs.0.bias depthwise_conv2d/Bias (32,) torch.Size([32])
backbone1.3.f.0.convs.1.weight conv2d_1/Kernel (32, 1, 1, 32) torch.Size([32, 32, 1, 1])
backbone1.3.f.0.convs.1.bias conv2d_1/Bias (32,) torch.Size([32])
backbone1.3.f.1.convs.0.weight depthwise_conv2d_1/Kernel (1, 3, 3, 32) torch.Size([32, 1, 3, 3])
backbone1.3.f.1.convs.0.bias depthwise_conv2d_1/Bias (32,) torch.Size([32])
backbone1.3.f.1.convs.1.weight conv2d_2/Kernel (32, 1, 1, 32) torch.Size([32, 32, 1, 1])
backbone1.3.f.1.convs.1.bias conv2d_2/Bias (32,) torch.Size([32])
backbone1.3.f.2.convs.0.weight depthwise_conv2d_2/Kernel (1, 3, 3, 32) torch.Size([32, 1, 3, 3])
backbone1.3.f.2.convs.0.bias depthwise_conv2d_2/Bias (32,) torch.Size([32])
backbone1.3.f.2.convs.1.weight conv2d_3/Kernel (32

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  app.launch_new_instance()


backbone2.1.f.0.convs.0.bias depthwise_conv2d_24/Bias (256,) torch.Size([256])
backbone2.1.f.0.convs.1.weight conv2d_25/Kernel (256, 1, 1, 256) torch.Size([256, 256, 1, 1])
backbone2.1.f.0.convs.1.bias conv2d_25/Bias (256,) torch.Size([256])
backbone2.1.f.1.convs.0.weight depthwise_conv2d_25/Kernel (1, 3, 3, 256) torch.Size([256, 1, 3, 3])
backbone2.1.f.1.convs.0.bias depthwise_conv2d_25/Bias (256,) torch.Size([256])
backbone2.1.f.1.convs.1.weight conv2d_26/Kernel (256, 1, 1, 256) torch.Size([256, 256, 1, 1])
backbone2.1.f.1.convs.1.bias conv2d_26/Bias (256,) torch.Size([256])
backbone2.1.f.2.convs.0.weight depthwise_conv2d_26/Kernel (1, 3, 3, 256) torch.Size([256, 1, 3, 3])
backbone2.1.f.2.convs.0.bias depthwise_conv2d_26/Bias (256,) torch.Size([256])
backbone2.1.f.2.convs.1.weight conv2d_27/Kernel (256, 1, 1, 256) torch.Size([256, 256, 1, 1])
backbone2.1.f.2.convs.1.bias conv2d_27/Bias (256,) torch.Size([256])
backbone2.1.f.3.convs.0.weight depthwise_conv2d_27/Kernel (1, 3, 3, 256) t

In [12]:
net.load_state_dict(new_state_dict, strict=True)

<All keys matched successfully>

In [13]:
torch.save(net.state_dict(), "palmdetector.pth")